In [3]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
import os
import dotenv

dotenv.load_dotenv()

True

In [12]:
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.getenv("groqKey"))

In [4]:
def calcular_imposto_renda(rendimento:float) -> str:
    """calcular o imposto de renda com base no rendimento anual

    args:
        rendimento: float - rendimento anual do individuo.
    returns:
        str - imposto de renda devido com base no rendimento   
    """

    if rendimento <= 2000:
        return "voce esta isento de pagar imposto de renda"
    
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"o imposto devido e de R${imposto:.2f}, base em um rendimento de R${rendimento:.2f}"
    
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"o imposto devido e de R${imposto:.2f}, base em um rendimento de R${rendimento:.2f}"
    
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"o imposto devido e de R${imposto:.2f}, base em um rendimento de R${rendimento:.2f}"

### convertendo função em ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

ferramento_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="calcular imposto de renda",
    description=("calcula o imposto de renda devido com base no rendimento anual"
                 "argumento:rendimento(float)"
                 "retorna o valor do imposto devido de acordo com faixas de rendimento"
                 )
    )

In [7]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [8]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramento_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [9]:
from llama_index.core.agent import AgentRunner

In [10]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [11]:
response = agent_imposto.chat("""
qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7.500?
""")

Added user message to memory: 
qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7.500?

=== Calling Function ===
Calling function: calcular imposto de renda with args: {"rendimento": 7500.0}
=== Function Output ===
o imposto devido e de R$675.00, base em um rendimento de R$7500.00
=== LLM Response ===
O imposto de renda devido por uma pessoa com rendimento anual de R$ 7.500 é de R$ 675,00.


In [12]:
import arxiv

def consulta_artigos(title:str) ->str:
    """consulta os artigos na base de dados arxiv e retorna
     resultado formatados."""
    
    busca = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"titiulo : {artigo.title}\n"
        f"categoria: {artigo.primary_category}\n"
        f"link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)

In [13]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
)

In [14]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[ferramento_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [16]:
agent = AgentRunner(agent_worker)

response = agent.chat("""
me retorne artigos sobre langchain na educação.
""")

Added user message to memory: 
me retorne artigos sobre langchain na educação.

=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Langchain na educa\u00e7\u00e3o"}


/tmp/ipykernel_13246/3869645632.py:17: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
titiulo : Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report
categoria: cs.CL
link: http://arxiv.org/abs/2402.01733v1


titiulo : From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
categoria: cs.CR
link: http://arxiv.org/abs/2308.01990v4


titiulo : Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
categoria: cs.CL
link: http://arxiv.org/abs/2310.05421v1


titiulo : Poisoned LangChain: Jailbreak LLMs by LangChain
categoria: cs.CL
link: http://arxiv.org/abs/2406.18122v1


titiulo : Breast Ultrasound Report Generation using LangChain
categoria: eess.IV
link: http://arxiv.org/abs/2312.03013v1

=== LLM Response ===
Aqui estão alguns artigos sobre Langchain na educação:

1. "Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report" (categoria: cs.CL) - Este arti

### Usando o tavily

In [6]:
tavily_key = os.getenv("tavily")

In [2]:
from llama_index.tools.tavily_research import TavilyToolSpec

In [7]:
tavily_tool = TavilyToolSpec(
    api_key=tavily_key,
)

In [8]:
tavily_tool_list = tavily_tool.to_tool_list()

for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [9]:
tavily_tool.search("me retorne artigos cientificos sobre langchain na educação",max_results=3)

[Document(id_='e515757d-16da-4bda-a1af-7e10d920075f', embedding=None, metadata={'url': 'https://www.codemag.com/Article/2501051/Exploring-LangChain-A-Practical-Approach-to-Language-Models-and-Retrieval-Augmented-Generation-RAG'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="In this article, I walked you through the essential components and practical applications of the LangChain framework. It starts with a basic example to establish a foundation, then explores chaining components, managing conversation memory, and using LangChain's memory features to stay within LLM context limits. Additionally, it covers language translation prompts, alternative models beyond OpenAI, and a hands-on guide to implementing Retrieval-Augmented Generation (RAG) for document-based [...] querying. With clear steps on chunking, creating retriever objec

In [10]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="tavily search",
    description="busca artigos com tavily sobre um determinado topico"
)

In [14]:
from llama_index.core.agent import FunctionCallingAgentWorker,AgentRunner
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    llm=llm,
    allow_parallel_tool_calls=False
)

In [15]:
agent = AgentRunner(agent_worker)

In [17]:
response = agent.chat("me retorne artigos cientificos sobre langchain na educação")

Added user message to memory: me retorne artigos cientificos sobre langchain na educação
=== Calling Function ===
Calling function: tavily search with args: {"query": "Langchain na educa\u00e7\u00e3o", "max_results": 10}
=== Function Output ===
[Document(id_='6bf6f4fe-be12-483d-ae67-82ee4e9ca0f3', embedding=None, metadata={'url': 'https://www.couchbase.com/pt/resources/concepts/what-is-langchain/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='A LangChain é uma plataforma avançada projetada para equipar os desenvolvedores com ferramentas essenciais para a criação de aplicativos orientados por modelos', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'), Document(id_='bf48be37-bc4e-407d-bd9e-658cb73b3f5f', embedding=None, metadata={'url'

In [6]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex

In [7]:
url = "LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()


Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [8]:
url = "LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

### Gerar os embeddings

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [4]:
Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

In [9]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [10]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

### engine de busca

In [11]:
from llama_index.core import StorageContext,load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='artigo')
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(persist_dir='tutorial')
tutorial_index = load_index_from_storage(storage_context)

In [13]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3,llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3,llm=llm)

In [15]:
from llama_index.core.tools import QueryEngineTool,ToolMetadata

In [16]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description="fornece informações sobre llm e langchain."
            "use uma pergunta detalhada em texto simples como entrada para a ferramenta"
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description="fornece informações sobre casos de uso e apliações em LLM"
            "use uma pergunta detalhada em texto simples como entrada para a ferramenta"
        )
    )
]

In [18]:
from llama_index.core.agent import FunctionCallingAgentWorker,AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [19]:
response = agent_document.chat("quais as principais apliações posso construir com LLM E langchain?")

Added user message to memory: quais as principais apliações posso construir com LLM E langchain?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "quais as principais aplica\u00e7\u00f5es posso construir com LLM e Langchain?"}
=== Function Output ===
As principais aplicações que você pode construir com LLMs incluem geração de texto, resumo, classificação e até mesmo geração de código. Além disso, você pode usar LLMs para criar chatbots, como o ChatGPT, que podem realizar tarefas complexas, como responder a perguntas e gerar conteúdo criativo. Com a combinação de LLMs e ferramentas como o Langchain, você pode criar aplicações personalizadas que atendam às suas necessidades específicas, como modelos de linguagem treinados com seus próprios dados. Isso pode ser útil para melhorar o desempenho dos modelos em seu domínio específico e manter os dados sob seu controle para preocupações com privacidade e governança.
=== Calling Function ===
Calling function: tutori

KeyboardInterrupt: 

In [21]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [22]:
response = agent.chat("quais as principais ferramentas usadas em langchain?")

> Running step 13257d65-d90e-4601-8be7-f4660d58d81a. Step input: quais as principais ferramentas usadas em langchain?
Thought: A pergunta está relacionada às ferramentas usadas em LangChain, então preciso usar uma ferramenta para fornecer informações sobre LangChain.
Action: artigo_engine
Action Input: {'input': 'quais as principais ferramentas usadas em LangChain'}
Observation: Não há menção às principais ferramentas usadas em LangChain no contexto fornecido. O contexto discute grandes modelos de linguagem (LLM), serviços proprietários, modelos de código aberto e o desenvolvimento histórico dos LLMs, mas não menciona LangChain ou suas ferramentas.
> Running step 03bcc75b-83af-4f26-b2cf-1c5249ab8cc2. Step input: None
Thought: A ferramenta artigo_engine não forneceu informações suficientes sobre as principais ferramentas usadas em LangChain, então preciso tentar novamente com uma pergunta mais específica.
Action: tutorial_engine
Action Input: {'input': 'quais são as principais ferrament